<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/EventProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QSTK

In [2]:
!pip install pandas_datareader

<h3> Fetching all tickers from sp500</h2>

In [9]:
def get_sp500_tickers():
  import bs4 as bs    
  import pickle    
  import requests    
  import lxml    
  resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')        
  soup = bs.BeautifulSoup(resp.text,'lxml')        
  table = soup.find('table', {'class': 'wikitable sortable'})        

  tickers = []

  for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[1].text
    tickers.append(str(ticker))

  return tickers    

res = get_sp500_tickers()
res

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'APC',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BBT',
 'BDX',
 'BRK-B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BR',
 'BF-B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CE',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS

<h2>Writing code to fetch data from yahoo (use IEX for recent data) </h2>

In [0]:
def get_data_from_yahoo(start, end, ticker, source='iex'):
  
  import pandas_datareader.data as dr
  import numpy as np
  from datetime import datetime
  try:
    return dr.DataReader(ticker, source, start, end)[['close']].rename(columns={'close': ticker})
  except Exception as e:
    print 'Exception when retrieving data for %s' % ticker



<h2> Importing required lbraries.(This code is borrowed from Computational Investing. course code only works for past data)</h2>

In [0]:


import pandas as pd; pd.TimeSeries = pd.Series


import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep


<h2> Importing functions from tutorial3 to reuse event profiler </h2>

In [0]:
def find_events(ls_symbols, d_data):
    ''' Finding the event dataframe '''
    df_close = d_data #['close']
    #ts_market = df_close['SPY']
    # Got it. we need to replace this with pandas.datareader
    print "Finding Events"

    # Creating an empty dataframe
    df_events = copy.deepcopy(df_close)
    df_events = df_events * np.NAN

    # Time stamps for the event range
    ldt_timestamps = df_close.index

    print 'Loopingthrough all symbols....'
    for s_sym in ls_symbols:
        try:
          print 'loop'
          for i in range(1, len(ldt_timestamps)):
              # Calculating the returns for this timestamp
              print 'I is  {}'.format(i)
              print df_close.columns
              f_symprice_today = df_close[s_sym].ix[ldt_timestamps[i]]
              print 'Todays price is:{}'.format(f_symprice_today)
              f_symprice_yest = df_close[s_sym].ix[ldt_timestamps[i - 1]]
              #f_marketprice_today = ts_market.ix[ldt_timestamps[i]]
              #f_marketprice_yest = ts_market.ix[ldt_timestamps[i - 1]]
              #f_symreturn_today = (f_symprice_today / f_symprice_yest) - 1
              #f_marketreturn_today = (f_marketprice_today / f_marketprice_yest) - 1
              
              # Event is found if the symbol is down more then 3% while the
              # market is up more then 2%
              #if f_symreturn_today <= -0.03 and f_marketreturn_today >= 0.02:
              # checking impact of 10% drop
              if f_symprice_today <= f_symprice_yest * 0.9:
                  print 'Found event for  {}'.format(s_sym)
                  df_events[s_sym].ix[ldt_timestamps[i]] = 1
        except Exception as e:
            print 'Exception %s ' % str(e)
          

    return df_events


In [0]:
def kick_off_events(start_date, end_date):
    ls_symbols = get_sp500_tickers()[0:1]
    ls_symbols.append('SPY')
    print 'About to get data for {}'.format(','.join(ls_symbols))
    ls_keys = ['close', 'volume']
    ldf_data = [get_data_from_yahoo(start_date, end_date, ticker) for ticker in ls_symbols]
    from pprint import pprint
    d_data = dict(zip(ls_symbols, ldf_data))

    
    for s_key in ls_symbols:
        try: 
          d_data[s_key] = d_data[s_key].fillna(method='ffill')
          d_data[s_key] = d_data[s_key].fillna(method='bfill')
          d_data[s_key] = d_data[s_key].fillna(1.0)
        except Exception as e:
          print 'Exception in dealing with %s data' % s_key

    print 'Finding event snow.....'
    print d_data.keys()
    '''
    df_events = find_events(ls_symbols, d_data)
    print "Creating Study for {}".format(len(df_events))
    print df_events.info()
    df_events.to_csv('EventProfile%s.csv')
    ep.eventprofiler(df_events, d_data, i_lookback=20, i_lookforward=20,
                s_filename='MyEventStudy-%s.pdf' , b_market_neutral=True, b_errorbars=True,
                s_market_sym='SPY')
    '''
  
  
  
  

In [35]:
from datetime import date
res = get_data_from_yahoo(date(2019,1,10), date(2019,1,30), 'SPY')
res

kick_off_events(date(2018,1,1), date(2018,12,31))


About to get data for MMM,SPY
                 MMM
date                
2018-01-02  229.5506
2018-01-03  229.5408
2018-01-04  232.5413
2018-01-05  234.3532
2018-01-08  233.5933
2018-01-09  235.0448
2018-01-10  234.9085
2018-01-11  236.0482
2018-01-12  238.1524
2018-01-16  238.4154
2018-01-17  241.6983
2018-01-18  239.5552
2018-01-19  241.7665
2018-01-22  240.9093
2018-01-23  239.9449
2018-01-24  241.2892
2018-01-25  245.8385
2018-01-26  251.9465
2018-01-29  249.3942
2018-01-30  245.0397
2018-01-31  244.0266
2018-02-01  241.5327
2018-02-02  238.8343
2018-02-05  225.4591
2018-02-06  227.5925
2018-02-07  227.1639
2018-02-08  217.1301
2018-02-09  219.3901
2018-02-12  223.2673
2018-02-13  225.3812
...              ...
2018-11-15  203.5201
2018-11-16  207.5824
2018-11-19  204.1856
2018-11-20  199.1499
2018-11-21  198.6900
2018-11-23  196.8000
2018-11-26  199.8600
2018-11-27  200.6900
2018-11-28  204.3400
2018-11-29  204.5600
2018-11-30  207.9200
2018-12-03  208.7600
2018-12-04  202.2000
2018